# Data Exploration

This notebook explores the collected market data to find patterns for scalping strategies.

**Prerequisites:**
- Run `scripts/collect_data.py` for at least a few hours
- Data stored in `data/raw/market_data.db`

In [ ]:
import sys
sys.path.insert(0, '..')

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from decimal import Decimal

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Data

In [ ]:
# Connect to database
DB_PATH = '../data/raw/market_data.db'
conn = sqlite3.connect(DB_PATH)

# Check tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tables in database:")
print(tables)

In [ ]:
# Load trades
trades_df = pd.read_sql("""
    SELECT symbol, trade_id, price, quantity, timestamp, is_buyer_maker
    FROM trades
    ORDER BY timestamp DESC
    LIMIT 100000
""", conn)

trades_df['price'] = trades_df['price'].astype(float)
trades_df['quantity'] = trades_df['quantity'].astype(float)
trades_df['timestamp'] = pd.to_datetime(trades_df['timestamp'])
trades_df['value'] = trades_df['price'] * trades_df['quantity']
trades_df['side'] = trades_df['is_buyer_maker'].map({0: 'BUY', 1: 'SELL'})

print(f"Loaded {len(trades_df):,} trades")
trades_df.head()

In [ ]:
# Load order book snapshots
orderbook_df = pd.read_sql("""
    SELECT symbol, timestamp, mid_price, spread_bps, imbalance_5,
           bid_volume_5, ask_volume_5
    FROM orderbook_snapshots
    ORDER BY timestamp DESC
    LIMIT 100000
""", conn)

orderbook_df['timestamp'] = pd.to_datetime(orderbook_df['timestamp'])
for col in ['mid_price', 'spread_bps', 'imbalance_5', 'bid_volume_5', 'ask_volume_5']:
    orderbook_df[col] = orderbook_df[col].astype(float)

print(f"Loaded {len(orderbook_df):,} order book snapshots")
orderbook_df.head()

In [ ]:
# Load mark prices / funding
funding_df = pd.read_sql("""
    SELECT symbol, timestamp, mark_price, index_price, funding_rate
    FROM mark_prices
    ORDER BY timestamp DESC
    LIMIT 10000
""", conn)

funding_df['timestamp'] = pd.to_datetime(funding_df['timestamp'])
for col in ['mark_price', 'index_price', 'funding_rate']:
    funding_df[col] = funding_df[col].astype(float)

print(f"Loaded {len(funding_df):,} mark price records")
funding_df.head()

## 2. Basic Statistics

In [ ]:
# Data timespan
if len(trades_df) > 0:
    print(f"Trades timespan: {trades_df['timestamp'].min()} to {trades_df['timestamp'].max()}")
    print(f"Duration: {trades_df['timestamp'].max() - trades_df['timestamp'].min()}")
else:
    print("No trades data yet. Run collect_data.py first!")

In [ ]:
# Trade statistics
if len(trades_df) > 0:
    print("Trade Statistics:")
    print(f"  Total trades: {len(trades_df):,}")
    print(f"  Total volume: ${trades_df['value'].sum():,.2f}")
    print(f"  Avg trade size: ${trades_df['value'].mean():.2f}")
    print(f"  Buy trades: {(trades_df['side'] == 'BUY').sum():,}")
    print(f"  Sell trades: {(trades_df['side'] == 'SELL').sum():,}")
    print(f"\nPrice range: ${trades_df['price'].min():.2f} - ${trades_df['price'].max():.2f}")

In [ ]:
# Order book statistics
if len(orderbook_df) > 0:
    print("Order Book Statistics:")
    print(f"  Snapshots: {len(orderbook_df):,}")
    print(f"  Avg spread: {orderbook_df['spread_bps'].mean():.2f} bps")
    print(f"  Avg imbalance: {orderbook_df['imbalance_5'].mean():.2f}")
    print(f"  Imbalance std: {orderbook_df['imbalance_5'].std():.2f}")

## 3. Price Movement Analysis

In [ ]:
# Resample trades to 1-second bars
if len(trades_df) > 0:
    trades_df = trades_df.sort_values('timestamp')
    trades_df.set_index('timestamp', inplace=True)
    
    bars_1s = trades_df.resample('1s').agg({
        'price': ['first', 'last', 'max', 'min'],
        'quantity': 'sum',
        'value': 'sum',
        'trade_id': 'count'
    }).dropna()
    
    bars_1s.columns = ['open', 'close', 'high', 'low', 'volume', 'value', 'trades']
    bars_1s['return'] = bars_1s['close'].pct_change() * 100  # in %
    
    print(f"Created {len(bars_1s):,} 1-second bars")
    bars_1s.head()

In [ ]:
# Plot price and volume
if len(trades_df) > 0 and len(bars_1s) > 100:
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    
    # Price
    axes[0].plot(bars_1s.index, bars_1s['close'], linewidth=0.5)
    axes[0].set_ylabel('Price ($)')
    axes[0].set_title('BTCUSDT Price')
    
    # Volume
    axes[1].bar(bars_1s.index, bars_1s['value'], width=0.00001, alpha=0.7)
    axes[1].set_ylabel('Volume ($)')
    axes[1].set_title('Trade Volume')
    
    # Returns
    axes[2].plot(bars_1s.index, bars_1s['return'], linewidth=0.5)
    axes[2].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[2].set_ylabel('Return (%)')
    axes[2].set_title('1-Second Returns')
    
    plt.tight_layout()
    plt.show()

## 4. Order Book Imbalance Analysis

In [ ]:
# Analyze imbalance vs future price movement
if len(orderbook_df) > 100:
    ob_df = orderbook_df.sort_values('timestamp').copy()
    ob_df.set_index('timestamp', inplace=True)
    
    # Calculate future returns (1s, 5s, 10s)
    ob_df['future_return_1s'] = ob_df['mid_price'].shift(-1) / ob_df['mid_price'] - 1
    ob_df['future_return_5s'] = ob_df['mid_price'].shift(-5) / ob_df['mid_price'] - 1
    ob_df['future_return_10s'] = ob_df['mid_price'].shift(-10) / ob_df['mid_price'] - 1
    
    # Categorize imbalance
    ob_df['imbalance_cat'] = pd.cut(
        ob_df['imbalance_5'], 
        bins=[0, 0.7, 0.9, 1.1, 1.3, float('inf')],
        labels=['Strong Sell', 'Weak Sell', 'Neutral', 'Weak Buy', 'Strong Buy']
    )
    
    # Analyze
    print("Average future return by imbalance category:")
    print(ob_df.groupby('imbalance_cat')[['future_return_1s', 'future_return_5s', 'future_return_10s']].mean() * 10000)  # in bps

In [ ]:
# Plot imbalance distribution
if len(orderbook_df) > 100:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Imbalance distribution
    axes[0].hist(orderbook_df['imbalance_5'].clip(0, 3), bins=50, edgecolor='black')
    axes[0].axvline(x=1.0, color='r', linestyle='--', label='Neutral')
    axes[0].set_xlabel('Imbalance (bid/ask volume)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Order Book Imbalance Distribution')
    axes[0].legend()
    
    # Spread distribution
    axes[1].hist(orderbook_df['spread_bps'].clip(0, 10), bins=50, edgecolor='black')
    axes[1].set_xlabel('Spread (bps)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Spread Distribution')
    
    plt.tight_layout()
    plt.show()

## 5. Volume Spike Analysis

In [ ]:
# Detect volume spikes
if len(trades_df) > 0 and len(bars_1s) > 60:
    bars = bars_1s.copy()
    
    # Rolling average volume (60 seconds)
    bars['avg_volume'] = bars['value'].rolling(60).mean()
    bars['volume_ratio'] = bars['value'] / bars['avg_volume']
    
    # Detect spikes (3x average)
    bars['is_spike'] = bars['volume_ratio'] > 3.0
    
    # Future returns after spikes
    bars['future_return_5s'] = bars['close'].shift(-5) / bars['close'] - 1
    bars['future_return_10s'] = bars['close'].shift(-10) / bars['close'] - 1
    
    spikes = bars[bars['is_spike']].copy()
    non_spikes = bars[~bars['is_spike']].copy()
    
    print(f"Total volume spikes detected: {len(spikes):,}")
    print(f"Spike percentage: {len(spikes)/len(bars)*100:.2f}%")
    print(f"\nAverage return after spike (5s): {spikes['future_return_5s'].mean()*10000:.2f} bps")
    print(f"Average return after spike (10s): {spikes['future_return_10s'].mean()*10000:.2f} bps")
    print(f"\nAverage return without spike (5s): {non_spikes['future_return_5s'].mean()*10000:.2f} bps")

## 6. Funding Rate Analysis

In [ ]:
# Analyze funding rate
if len(funding_df) > 0:
    funding = funding_df.sort_values('timestamp').copy()
    funding.set_index('timestamp', inplace=True)
    
    print("Funding Rate Statistics:")
    print(f"  Mean: {funding['funding_rate'].mean()*100:.4f}%")
    print(f"  Std: {funding['funding_rate'].std()*100:.4f}%")
    print(f"  Min: {funding['funding_rate'].min()*100:.4f}%")
    print(f"  Max: {funding['funding_rate'].max()*100:.4f}%")
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(funding.index, funding['funding_rate']*100)
    ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    ax.set_ylabel('Funding Rate (%)')
    ax.set_title('Funding Rate Over Time')
    plt.show()

## 7. Key Findings Summary

Run this section after collecting sufficient data (5-7 days) to document findings.

In [ ]:
# Summary statistics
print("=" * 50)
print("DATA COLLECTION SUMMARY")
print("=" * 50)

if len(trades_df) > 0:
    print(f"\nTrades: {len(trades_df):,}")
if len(orderbook_df) > 0:
    print(f"Order Book Snapshots: {len(orderbook_df):,}")
if len(funding_df) > 0:
    print(f"Mark Price Records: {len(funding_df):,}")

print("\n" + "=" * 50)
print("NEXT STEPS:")
print("=" * 50)
print("1. Collect more data (target: 5-7 days)")
print("2. Analyze imbalance threshold for signals")
print("3. Analyze volume spike patterns")
print("4. Backtest potential strategies")

In [ ]:
# Close database connection
conn.close()